# Pre-Processing and Training Data

In [149]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from library.sb_utils import save_file

In [150]:
# import data
wnv = pd.read_csv('../data/WestNileVirus_cleaned.csv')
wnv.head().T

,0,1,2,3,4
Trap,T001,T001,T001,T001,T001
Block,40,40,40,40,40
Latitude,41.953705,41.953705,41.953705,41.953705,41.953705
Longitude,-87.733974,-87.733974,-87.733974,-87.733974,-87.733974
Date,2007-06-26,2007-07-11,2007-07-18,2007-08-01,2007-08-01
Species,CULEX PIPIENS/RESTUANS,CULEX PIPIENS/RESTUANS,CULEX PIPIENS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS
AddressAccuracy,8,8,8,8,8
NumMosquitos,1,1,1,1,3
Tmax,91.5,77.0,85.0,91.5,91.5
Tmin,71.5,62.5,69.0,69.0,69.0


In [151]:
wnv.SnowFall.unique()

array([0.   , 0.001])

In [152]:
# convert 'Date' column to datetime
wnv['Date'] = pd.to_datetime(wnv['Date'], format="%Y/%m/%d")

In [153]:
wnv.dtypes

Trap                       object
Block                       int64
Latitude                  float64
Longitude                 float64
Date               datetime64[ns]
Species                    object
AddressAccuracy             int64
NumMosquitos                int64
Tmax                      float64
Tmin                      float64
Tavg                      float64
Depart                    float64
DewPoint                  float64
WetBulb                   float64
Heat                      float64
Cool                      float64
Sunrise                   float64
Sunset                    float64
SnowFall                  float64
PrecipTotal               float64
StnPressure               float64
SeaLevel                  float64
ResultSpeed               float64
ResultDir                 float64
AvgSpeed                  float64
WnvPresent                  int64
Month                       int64
Year                        int64
Day                         int64
dtype: object

In the EDA section, we determined that the `Heat`, `SnowFall` and `PrecipTotal` did not give us useful insights. We will be dropping these columns. 

We also split the `Date` column into individual `Year`, `Month` and `Day` columns so we can drop the `Date` column. We will also drop the `Day` column.

In [154]:
# drop 'Year' and 'Day' columns
columns_to_drop = ['Date', 'Heat', 'SnowFall', 'PrecipTotal', 'Day']
wnv = wnv.drop(columns_to_drop, axis=1)

### One-Hot Encode Categorical Columns

Next we want to one-hot encode the categorical variables.

In [155]:
categorical_cols = ['Trap', 'Block', 'Species', 'AddressAccuracy', 'Month', 'Year']

In [156]:
wnv = pd.get_dummies(wnv, columns=categorical_cols)

In [157]:
wnv.head().T

,0,1,2,3,4
Latitude,41.953705,41.953705,41.953705,41.953705,41.953705
Longitude,-87.733974,-87.733974,-87.733974,-87.733974,-87.733974
NumMosquitos,1.000000,1.000000,1.000000,1.000000,3.000000
Tmax,91.500000,77.000000,85.000000,91.500000,91.500000
Tmin,71.500000,62.500000,69.000000,69.000000,69.000000
...,...,...,...,...,...
Month_10,0.000000,0.000000,0.000000,0.000000,0.000000
Year_2007,1.000000,1.000000,1.000000,1.000000,1.000000
Year_2009,0.000000,0.000000,0.000000,0.000000,0.000000
Year_2011,0.000000,0.000000,0.000000,0.000000,0.000000


### Normalize Quatitative Columns

In [158]:
list(wnv.columns)

['Latitude',
 'Longitude',
 'NumMosquitos',
 'Tmax',
 'Tmin',
 'Tavg',
 'Depart',
 'DewPoint',
 'WetBulb',
 'Cool',
 'Sunrise',
 'Sunset',
 'StnPressure',
 'SeaLevel',
 'ResultSpeed',
 'ResultDir',
 'AvgSpeed',
 'WnvPresent',
 'Trap_T001',
 'Trap_T002',
 'Trap_T003',
 'Trap_T004',
 'Trap_T005',
 'Trap_T006',
 'Trap_T007',
 'Trap_T008',
 'Trap_T009',
 'Trap_T009A',
 'Trap_T011',
 'Trap_T012',
 'Trap_T013',
 'Trap_T014',
 'Trap_T015',
 'Trap_T016',
 'Trap_T017',
 'Trap_T018',
 'Trap_T019',
 'Trap_T025',
 'Trap_T027',
 'Trap_T028',
 'Trap_T030',
 'Trap_T031',
 'Trap_T033',
 'Trap_T034',
 'Trap_T035',
 'Trap_T035A',
 'Trap_T036',
 'Trap_T037',
 'Trap_T039',
 'Trap_T040',
 'Trap_T043',
 'Trap_T044',
 'Trap_T045',
 'Trap_T046',
 'Trap_T047',
 'Trap_T048',
 'Trap_T049',
 'Trap_T050',
 'Trap_T051',
 'Trap_T054',
 'Trap_T054C',
 'Trap_T060',
 'Trap_T061',
 'Trap_T062',
 'Trap_T063',
 'Trap_T065',
 'Trap_T066',
 'Trap_T067',
 'Trap_T069',
 'Trap_T070',
 'Trap_T071',
 'Trap_T072',
 'Trap_T073',
 

In [159]:
quant_columns = ['Latitude', 'Longitude', 'NumMosquitos', 'Tmax', 'Tmin', 'Tavg', 'Depart',
                 'DewPoint', 'WetBulb', 'Cool', 'Sunrise', 'Sunset', 'StnPressure',
                 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed',]

In [160]:
# scale continiuous variables
scaler = StandardScaler()
scaler.fit(wnv[quant_columns])
wnv[quant_columns] = scaler.transform(wnv[quant_columns])

### Train/Test Split

In [161]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(wnv.drop(columns='WnvPresent'), 
                                                    wnv.WnvPresent, test_size=0.3, 
                                                    random_state=47)

In [162]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6027, 240) (6027,) (2583, 240) (2583,)


### Save new .csv file

In [ ]:
# save data to a new .csv file
#datapath = '../data'
#save_file(wnv, 'wnv.csv', datapath)